## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various binary classifiers

In [ ]:
# Exploratory data analysis (regression)

In [4]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42
k_fold_splits = 2

np.random.seed(global_random_state)


active_pct = 0.073125471
inactive_pct = 1 - active_pct

# We set the inactive to have the weight of the active, and vice versa, to account for imbalance
class_weights = { 0: active_pct, 1: inactive_pct }

In [ ]:
# Load assay info. Note: This CSV was obtained from PubChem bioassay aka PCBA, via searching for AID 1030 
# and downloading the datatable

ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)
full_df["PUBCHEM_ACTIVITY_SCORE"] = full_df["PUBCHEM_ACTIVITY_SCORE"].astype(int)

# Normalize pubchem activity score to 0...1
full_df["PUBCHEM_ACTIVITY_SCORE"] = full_df["PUBCHEM_ACTIVITY_SCORE"] / 100


compound_ids = list()
smiles_list = list()
fingerprints = list()
activities = list()

#fingerprint_df = 

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    activity_score = row["PUBCHEM_ACTIVITY_SCORE"]
    if mol is None:
        print("Molecule failed featurization")
        print(index)
    else: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(activity_score)
    
    if index % 10000 == 0:
        print("Processed index: {0}".format(index))

# Convert activities to np array of ints

X = np.array(fingerprints)
y = np.array(activities,dtype=float)

print(X[0:10])
print(y[0:10])

# Pickle the data to save time in the future
with open('data.regression.undersampled.pickle', 'wb') as f:
    pickle.dump((X,y), f, pickle.HIGHEST_PROTOCOL)


Processed index: 0
Processed index: 10000
Processed index: 20000
Processed index: 30000
Processed index: 40000
Processed index: 50000
Processed index: 60000
Processed index: 70000
Processed index: 80000
Processed index: 90000
Processed index: 100000
Processed index: 110000


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import cross_val_predict
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from collections import Counter
from sklearn.tree import DecisionTreeRegressor

smiles_list = None
compound_ids = None
fingerprints = None
activities = None

global_random_state = 42

with open('data.regression.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

# Print the number of compounds loaded
print("Successfully loaded {0} compounds.".format(len(X)))

#print(X.shape)
#print(y.shape)

#X = np.reshape(X,(len(X),))
#y = np.reshape(y,(len(y),))

# Create a train/test split

kf = KFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

mse_avg = 0

for train_index, test_index in kf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print(X_train[0:10])
    print(y_train[0:10])

    classifier = DecisionTreeRegressor(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Computed mse score of: {}".format(mse))
    print(classification_report(y_test, y_pred))
    mse_avg = mse_avg + mse
    
    y_pred_binary = y_pred > .4

    print(y_pred_binary[0:100])

    y_test_binary = y_test[0:] > .4

    print(y_test_binary[0:100])

    print("How good is it as a classifier?")
    print(classification_report(y_test_binary, y_pred_binary))
    
mse_avg = mse_avg / k_fold_splits
print("Average mse score is: {}".format(mse_avg))

# Note: Unfortunately it's not directly comparable to ROC_AUC calculated in MoleculeNet at: https://arxiv.org/pdf/1703.00564.pdf 
# This is because MoleculeNet looks at a different metric (roc_auc) and also a different task (multiclass prediction across 128 bioassays simultaneously vs binary classification here)

In [2]:
import keras
print(keras.backend.backend())

Using TensorFlow backend.


tensorflow


In [ ]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


global_random_state = 42

with open('data.regression.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (compound_ids, smiles_list, fingerprints, activities) = pickle.load(f)

def create_model() :
    model = Sequential()
    model.add(Dense(12, input_dim=2048, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.binary_accuracy])
    return model

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=1)
kfold = StratifiedKFold(n_splits=k_fold_splits, shuffle=True, random_state=global_random_state)
results = cross_val_score(model, fingerprints, activities, cv=k_fold_splits)
print(results.mean())

y_pred = cross_val_predict(classifier, fingerprints, activities, cv=k_fold_splits)

print(classification_report(activities, y_pred))

In [ ]:
y_pred = model.predict_on_batch(X_test)
y_pred_binarized = y_pred[0:] > .5
print(classification_report(y_test, y_pred_binarized))


In [ ]:
# What about a larger network size?
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics

model = Sequential()
model.add(Dense(1024, input_dim=2048, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.binary_accuracy])

model.fit(X_train, y_train_, epochs=10, batch_size=1)

score = model.evaluate(X_test, y_test_binary)

print("\n Loss on test set is: {0}".format(score))

# let's save it for future experimentation
model.save("pcba_1030_large_nn.h5")

In [ ]:
from keras.models import load_model
model_large = load_model("pcba_1030_large_nn.h5")
y_pred = model_large.predict_on_batch(X_test)
y_pred_binarized = y_pred[0:] > .5
print(classification_report(y_test, y_pred_binarized))

In [ ]:
# Does an MLP classifier help?
# Answer: Nope, not really

from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(random_state=global_random_state,class_weight=class_weights)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))



In [5]:
# What if we train directly on the full (unbalanced) dataset and also set radius of 1?

import pandas as pd
import numpy as np
from collections import Counter

import pickle

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter


ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)

full_df["Active"] = full_df["PUBCHEM_ACTIVITY_OUTCOME"] == True


compound_ids = list()
smiles_list = list()
fingerprints = list()
activities = list()

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    if mol is not None: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,1,nBits=4096,useChirality=True,
                                                                     useBondTypes=True,useFeatures=True)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(is_active)



/home/joe/anaconda3/envs/pubchem-sklearn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,8,9,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
import pickle

compound_ids_and_features = (compound_ids, smiles_list, fingerprints, activities)

# Pickle the data to save time in the future
with open('data_large.pickle', 'wb') as f:
    pickle.dump(compound_ids_and_features, f, pickle.HIGHEST_PROTOCOL)
        

In [ ]:
# Let's try using a Random forest

classifier = RandomForestClassifier(n_estimators=100, random_state=global_random_state, n_jobs=-1)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test, class_weight=class_weights)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))


In [1]:
import pickle
with open('data_large.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (compound_ids, smiles_list, fingerprints, activities) = pickle.load(f)

print("Data loaded")


Data loaded


In [ ]:
# What about a larger network size, and setting class weights?
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
import pickle
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler


global_random_state = 42

#activity_outcomes = pd.Series(activities).astype(bool)

#X_train, X_test, y_train, y_test = train_test_split(np.array(fingerprints), np.array(activities, dtype=int), test_size=0.33, random_state=global_random_state)

active_pct = 0.073125471
inactive_pct = 1 - active_pct

# We set the inactive to have the weight of the active, and vice versa, to account for imbalance
class_weights = { 0: active_pct, 1: inactive_pct }

print(len(X_train))
print(len(X_train[0]))
print(y_train[0])

#y_train_binary = np.array(y_train,dtype=np.bool)
#y_test_binary = np.array(y_test,dtype=np.bool)


model = Sequential()
model.add(Dense(2048, input_dim=2048, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.binary_accuracy])
model.fit(X_train, y_train, epochs=10, batch_size=1, class_weight=class_weights)

score = model.evaluate(X_test, y_test)

print("\n Loss on test set is: {0}".format(score))

# let's save it for future experimentation
model.save("pcba_1030_large_nn.h5")

y_pred = model.predict_on_batch(X_test)
y_pred_binarized = y_pred[0:] > .6
print(classification_report(y_test, y_pred_binarized))

147643
2048
0
Epoch 1/5
  2702/147643 [..............................] - ETA: 579s - loss: 1.8638 - binary_accuracy: 0.6029

In [ ]:
# What about a larger network size, and setting class weights?
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
import pickle
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler


global_random_state = 42

#activity_outcomes = pd.Series(activities).astype(bool)

#X_train, X_test, y_train, y_test = train_test_split(np.array(fingerprints), np.array(activities, dtype=int), test_size=0.33, random_state=global_random_state)

active_pct = 0.073125471
inactive_pct = 1 - active_pct

# We set the inactive to have the weight of the active, and vice versa, to account for imbalance
class_weights = { 0: active_pct, 1: inactive_pct }

print(len(X_train))
print(len(X_train[0]))
print(y_train[0])

#y_train_binary = np.array(y_train,dtype=np.bool)
#y_test_binary = np.array(y_test,dtype=np.bool)


model = Sequential()
model.add(Dense(2048, input_dim=2048, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.binary_accuracy])
model.fit(X_train, y_train, epochs=10, batch_size=1, class_weight=class_weights)

score = model.evaluate(X_test, y_test)

print("\n Loss on test set is: {0}".format(score))

# let's save it for future experimentation
model.save("pcba_1030_large_nn.h5")

y_pred = model.predict_on_batch(X_test)
y_pred_binarized = y_pred[0:] > .6
print(classification_report(y_test, y_pred_binarized))

In [9]:
y_pred = model.predict_on_batch(X_test)
y_pred_binarized = y_pred[0:] > .4
print(classification_report(y_test, y_pred_binarized))

             precision    recall  f1-score   support

          0       0.96      0.83      0.89     67409
          1       0.20      0.54      0.30      5312

avg / total       0.90      0.81      0.85     72721



In [2]:
# Correctly balance the classes
import sklearn
from imblearn.over_sampling import RandomOverSampler

global_random_state = 42
#rus = RandomUnderSampler(random_state=global_random_state)
X_resampled, y_resampled = ros.fit_sample(fingerprints, activities)

#print(sorted(Counter(y_resampled).items()))


#X_train, X_test, y_train, y_test = train_test_split(X_resampled, 
                                                    y_resampled, test_size=0.33, random_state=global_random_state)

#print(X_train.head())
#print(y_train.head())

print(len(X_train))
print(len(y_train))

classifier = RandomForestClassifier(n_estimators=100, random_state=global_random_state, n_jobs=-1)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))


[(False, 16111), (True, 16111)]


AttributeError: 'numpy.ndarray' object has no attribute 'head'

#### 

In [1]:
# Let's try again. We duplicate imports in case starting from this cell

import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

smiles_list = None
compound_ids = None
fingerprints = None
activities = None

global_random_state = 42

with open('data.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (compound_ids, smiles_list, fingerprints, activities) = pickle.load(f)

# Print the number of compounds loaded
print("Successfully loaded {0} compounds.".format(len(compound_ids)))

# Show an example of the raw data
print("Sample ID: {0}".format(compound_ids[0]))
print("Sample fingerprint vector: {0}".format(fingerprints[0]))
print("Was it bioactive? Assay returned: {0}".format(activities[0]))

# Correctly balance the classes

undersample = False
oversample = False
if undersample :
    rus = RandomUnderSampler(random_state=global_random_state)
    X_resampled, y_resampled = rus.fit_sample(fingerprints, activities)
elif oversample :
    ros = RandomOverSampler(random_state=global_random_state)
    X_resampled, y_resampled = rus.fit_sample(fingerprints, activities) 
else :
    X_resampled = fingerprints
    y_resampled = activities
    
print(sorted(Counter(y_resampled).items()))

X_train, X_test, y_train, y_test = train_test_split(np.array(X_resampled), np.array(y_resampled, dtype=int), test_size=0.33, random_state=global_random_state)

Successfully loaded 220364 compounds.
Sample ID: 6603008
Sample fingerprint vector: [ 0.  0.  0. ...,  0.  0.  0.]
Was it bioactive? Assay returned: False
[(False, 204253), (True, 16111)]


In [ ]:
# Create a new train/test split based on resampled data

classifier = DecisionTreeClassifier(random_state=global_random_state)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))
